# Visiting Shrewsbury

How many times did I visit Shrewsbury?  Or at least how many times did my telephone visit Shrewsbury?

This is based on this strategy <https://shiring.github.io/maps/2016/12/30/Standortverlauf_post> for querying the location data.

Google Take Out <https://takeout.google.com/settings/takeout> provides my location data in Json format, with `jsonlite` will load.  The `Location History.json` is already stripped down to 7 days, so there is less of it to handle.  The whole file is pretty big so is difficult to work with on Azure Notebooks.

In [1]:
# Load the package
library(jsonlite)
system.time(import <- fromJSON("Location History.json"))

   user  system elapsed 
  0.882   0.011   1.286 

In [2]:
# extracting the locations dataframe
locations = import$locations
head(locations[,c("latitudeE7", "longitudeE7", "timestampMs")], n=3)

latitudeE7,longitudeE7,timestampMs
525622196,-27951121,1541616455811
525622196,-27951121,1541616049051
525622196,-27951121,1541615881157


## Converting the Data

The dates are held as POSIX dates (milliseconds since the begining of time) and need to be converted to to queried in a readable manner.  Also the locations are held as integers (for convenience) so need to be divided by 10^7 to get the latitude and longitudes.  The 'E7' on the field implies this.

In [3]:
# converting time column from posix milliseconds into a readable time scale
locations$date_time = as.POSIXct(as.numeric(locations$timestampMs)/1000, origin = "1970-01-01")
locations$date <- as.Date(locations$date_time)
# converting longitude and latitude from E7 to GPS coordinates
locations$lat = locations$latitudeE7 / 1e7
locations$lon = locations$longitudeE7 / 1e7

# To get the days of the week so I can query for thing 'weekend' install and load the package `lubridate`
install.packages("lubridate")
library(lubridate)

locations$weekday <- wday(as.Date(locations$date_time))
head(locations[,c("lat", "lon", "date", "weekday")], n=3) 

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("lubridate"):
“installation of package ‘lubridate’ had non-zero exit status”
Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



lat,lon,date,weekday
52.56222,-2.795112,2018-11-07,4
52.56222,-2.795112,2018-11-07,4
52.56222,-2.795112,2018-11-07,4


## Select the Visits

Using a box around Shrewsbury where:

* bottom left = 52.704117, -2.762084
* top right = 52.709638, -2.748738 

And selecting the where the weekday is between 2 and 6 (Sunday = 1 and Saturday = 7, don't ask me why).

In [4]:
locations_in_shrewsbury <- subset(locations, lat > 52.704117 & lat < 52.709638 & lon< -2.748738 & lon> -2.762084 & weekday > 1 & weekday < 7)
visits_to_shrewsbury_in_working_week <- unique(locations_in_shrewsbury[c("date","weekday")])
head(visits_to_shrewsbury_in_working_week, n=10)

,date,weekday
66,2018-11-07,4
451,2018-11-06,3
983,2018-11-02,6


Giving 3 visits to Shrewsbury in those 7 days.